In [3]:
import pandas as pd
import numpy as np
import torch
import time
import Resources.utils_stripped as U

In [4]:
train_df = pd.read_csv("Input/opp115_train.csv")
test_df = pd.read_csv("Input/opp115_test.csv")
train_df, test_df = U.load_opp115_from_df(train_df, test_df)
df_train, df_val = U.split_train_val(train_df, val_size=0.2)

# Training of Models

## RoBERTa Model

In [ ]:
res = U.train_transformer_mcc(
    model_type="roberta",
    df_train=df_train,
    df_val=df_val,
    epochs=3,
    batch_size=32,
    lr=2e-5,
    out_dir="artifacts/roberta"
)

model_roberta = res["model"]
tokenizer_roberta = res["tokenizer"]

# # inference
# pred_labels, pred_proba = U.predict_transformer(model_roberta, tokenizer_roberta, ["Some privacy sentence..."], return_proba=True)
# pred_labels[0]

## BERT Model

In [ ]:
# 3) Train BERT (multi-class)
res = U.train_transformer_mcc(
    model_type="bert",
    df_train=df_train,
    df_val=df_val,
    epochs=3,
    batch_size=32,
    lr=2e-5,
    out_dir="artifacts/bert_mcc"
)

bert_model = res["model"]
bert_tokenizer = res["tokenizer"]

# Inference
# pred_labels, pred_proba = U.predict_transformer(bert_model, bert_tokenizer, ["Some privacy sentence..."], return_proba=True)
# pred_labels[0]


## BiLSTM Model

In [5]:
df_train, df_val = U.split_train_val(train_df, val_size=0.2)

with U.measure_run("Train BiLSTM"):
    res = U.train_bilstm_mcc(
        df_train=df_train,
        df_val=df_val,
        epochs=3,
        batch_size=32,
        lr=1e-4,
        out_dir="artifacts/bilstm"
    )

bilstm = res["model"]

# # inference
# pred_labels, pred_proba = U.predict_bilstm(bilstm, ["Some privacy sentence..."], return_proba=True)
# pred_labels[0]


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:02<00:00, 37.43it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallel

=== PERF ===
tag: Train BiLSTM
wall_time_sec: 38.65933985599986
cpu_user_sec: 28.620000000000005
cpu_system_sec: 6.2700000000000005
ram_delta_mb: 735.3828125
gpu_peak_mem_mb: 659.55078125


# Evaluation of Models

## BiLSTM

In [9]:
print("test_df rows:", len(test_df))
print("train_df rows:", len(train_df))
print("Sentence non-null:", test_df["Sentence"].notna().sum())
print("Unique labels:", test_df["label"].nunique())

bilstm.eval()

sentences = test_df["Sentence"].tolist()
y_true = test_df["label"].astype(int).to_numpy()

with U.measure_run("BiLSTM inference"):
    y_pred_names, _ = U.predict_bilstm(bilstm, sentences, return_proba=True)

# map string labels → ids
CLASS_TO_ID = {name: i for i, name in enumerate(U.LABEL_NAMES)}
y_pred = np.array([CLASS_TO_ID[name] for name in y_pred_names], dtype=int)

results = U.evaluate_predictions(y_true, y_pred)

print("Confusion matrix:")
print(results["confusion_matrix"])

p = results["report"]["macro avg"]["precision"]
r = results["report"]["macro avg"]["recall"]

print("Macro F1:", results["report"]["macro avg"]["f1-score"])
print("Macro F2:", 0.0 if (4*p + r) == 0 else (5*p*r)/(4*p + r))

test_df rows: 2480
train_df rows: 14336
Sentence non-null: 2480
Unique labels: 9
=== PERF ===
tag: BiLSTM inference
wall_time_sec: 4.77458802899946
cpu_user_sec: 2.950000000000003
cpu_system_sec: 0.6100000000000012
ram_delta_mb: 0.68359375
gpu_peak_mem_mb: 625.11767578125
Confusion matrix:
[[782  55  46   0   0   0   0   0  20]
 [ 85 633   3   0   0   0   0   0  50]
 [ 57   4 162   0   0   0   0   0  45]
 [ 27   1   3   0   0   0   0   0   1]
 [  8   3   2   0   0   1   0   0  73]
 [  4   0   3   0   0   8   0   0  32]
 [  5   0   0   0   0   0   0   0   4]
 [  7   5   3   0   0   4   0   0  50]
 [ 39  15  45   0   0   3   0   0 192]]
Macro F1: 0.33682351871785
Macro F2: 0.3473634889639036
